BiRefNet HR Matting
====

**Bilateral Reference for High-Resolution Dichotomous Image Segmentation**

 * Paper: https://arxiv.org/abs/2401.03407

![BiRefNet Arch](../assets/birefnet_arch.png)

![BiRefNet Pipeline](../assets/birefnet_pipeline.png)

![BiRefNet Pipeline](../assets/birefnet_block.png)


```bash
conda create -n birefnet python=3.10 -y
conda activate birefnet

pip install -r https://raw.githubusercontent.com/ZhengPeng7/BiRefNet/main/requirements.txt

pip install transformers
pip install matplotlib
```

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from transformers import AutoModelForImageSegmentation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

birefnet = AutoModelForImageSegmentation.from_pretrained(
    'ZhengPeng7/BiRefNet_HR-matting',
    trust_remote_code=True
)
torch.set_float32_matmul_precision(['high', 'highest'][0])
birefnet.eval().to(device).half();

/home/pyml/anaconda3/envs/birefnet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pyml/anaconda3/envs/birefnet/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/pyml/anaconda3/envs/birefnet/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
